In [1]:
import os, psutil  

def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import time, gc

/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/train.csv
/kaggle/input/bengaliai-cv19/class_map_corrected.csv
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_multi_diacritics.csv
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet
/kaggle/input/kerasefficientnetb3/Train1_model_70.h5
/kaggle/input/kerasefficientnetb3/efficientnet-1.0.0-py3-none-any.whl
/kaggle/input/kerasefficientnetb3/Train1_model_64.h5
/kaggle/input/kerasefficientnetb3/Train1_model_57.h5
/kaggle/input/kerasefficientnetb3/Train1_model_

In [3]:
# 경로
DIR = '../input/bengaliai-cv19'
test_loc = 'test_image_data_'
#test_loc = 'train_image_data_'

In [4]:
import sys

import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler

#sys.path.insert(0, "/kaggle/input/ghostnet")
sys.path.insert(0, "/kaggle/input/ghostnetbengali")

from ghost_net import ghost_net

In [5]:
cpu_stats()

'memory GB:0.21'

In [6]:
IMAGE_SIZE = 137, 236


class DatasetRetriever(Dataset):
    
    def __init__(self, df):
        self.image_ids = df.iloc[:, 0].tolist()
        self.images = torch.from_numpy(255 - df[[str(x) for x in range(32332)]].values)

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        img = self.images[idx]
        img = img.view(*IMAGE_SIZE)
        img = img.to(torch.float32) / 255.
        img = img.unsqueeze(0)
        img = img.repeat(3, 1, 1)
        img_id = self.image_ids[idx]        
        return img_id, img
    
class Predictor:
    def __init__(self, model):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device, dtype=torch.float32);
        self.model.eval()
        print(f'Model prepared. Device is {self.device}')
    
    def predict(self, inputs):
        inputs = inputs.to(self.device, dtype=torch.float32)
        with torch.no_grad():
            outputs = self.model(inputs)
        return outputs

    def load(self, path):
        checkpoint = torch.load(path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])

In [7]:
cpu_stats()

'memory GB:0.21'

In [8]:
predictors = []

for i, model_path in enumerate([
    #'/kaggle/input/ghostnetbengali/checkpoint.pt',
    #'/kaggle/input/ghostnetbengali/checkpoint-15.pt',
    #'/kaggle/input/ghostnetbengali/checkpoint-16.pt',
    '/kaggle/input/ghostnetbengali/checkpoint-17.pt',
    '/kaggle/input/ghostnetbengali/checkpoint-18.pt',
]):
    predictor = Predictor(ghost_net(num_classes=168+11+7))
    predictor.load(model_path)
    predictors.append(predictor)

predictors_count = len(predictors)


def predict_to_numpy(predict):
    return torch.nn.functional.softmax(predict, dim=1).data.cpu().numpy()

def make_prediction(images):
    outputs = 0
    for predictor in predictors:
        outputs += predictor.predict(images) / predictors_count

    roots = predict_to_numpy(outputs[:,:168])
    vowels = predict_to_numpy(outputs[:,168:168+11])
    consonants = predict_to_numpy(outputs[:,168+11:])
    return roots, vowels, consonants

Model prepared. Device is cuda:0
Model prepared. Device is cuda:0


In [9]:
cpu_stats()

'memory GB:1.83'

In [10]:
%%time

target = []
row_id = []

roots_id_ghost=[]
roots_prob_ghost=[]

vowels_id_ghost=[]
vowels_prob_ghost=[]

consonants_id_ghost=[]
consonants_prob_ghost=[]

for i in range(4):
    data = pd.read_parquet(os.path.join(DIR, test_loc+str(i)+'.parquet'))
    #data = pd.read_parquet(f'../input/bengaliai-cv19/test_image_data_{i}.parquet')
    dataset = DatasetRetriever(data)
    data_loader = DataLoader(dataset, batch_size=256, num_workers=4, shuffle=False, sampler=SequentialSampler(dataset))

    for idx, (image_ids, images) in tqdm(enumerate(data_loader), total=len(data_loader)):

        roots, vowels, consonants = make_prediction(images)

        #for image_id, root, vowel, consonant in zip(image_ids, roots, vowels, consonants):
        for root, vowel, consonant in zip(roots, vowels, consonants):
            #원본
            #row_id.append(image_id + '_consonant_diacritic')
            #target.append(consonant)
            #row_id.append(image_id + '_grapheme_root')
            #target.append(root)
            #row_id.append(image_id + '_vowel_diacritic')
            #target.append(vowel)
            
            
            
            #roots_prob_ghost.append(roots)
            #vowels_prob_ghost.append(vowels)
            #consonants_prob_ghost.append(consonants)             
            
            
            roots_prob_ghost.append(root)
            vowels_prob_ghost.append(vowel)
            consonants_prob_ghost.append(consonant)    
            
    del data
    del dataset
    del data_loader
    collected = gc.collect()
    print(collected)
            
            
            
            
'''
xx=[0,3,6,9,12,15,18,21,24,27,30,33]
yy=[1,4,7,10,13,16,19,22,25,28,31,34]
zz=[2,5,8,11,14,17,20,23,26,29,32,35]

for i in xx:
    roots_prob_ghost.append(targets[i])
    
for i in yy:
    vowels_prob_ghost.append(targets[i])
    
for i in zz:
    consonants_prob_ghost.append(targets[i])
'''

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


0


100%|██████████| 1/1 [00:00<00:00,  8.21it/s]


2


100%|██████████| 1/1 [00:00<00:00,  8.97it/s]


2


100%|██████████| 1/1 [00:00<00:00,  8.52it/s]

2
CPU times: user 11.7 s, sys: 2.41 s, total: 14.1 s
Wall time: 12.6 s


'\nxx=[0,3,6,9,12,15,18,21,24,27,30,33]\nyy=[1,4,7,10,13,16,19,22,25,28,31,34]\nzz=[2,5,8,11,14,17,20,23,26,29,32,35]\n\nfor i in xx:\n    roots_prob_ghost.append(targets[i])\n    \nfor i in yy:\n    vowels_prob_ghost.append(targets[i])\n    \nfor i in zz:\n    consonants_prob_ghost.append(targets[i])\n'

In [11]:
cpu_stats()

'memory GB:2.22'

In [12]:
len(roots_prob_ghost[0])

168

In [13]:
np.array(roots_prob_ghost).shape

(12, 168)

In [14]:
vowels_prob_ghost

[array([9.9987340e-01, 1.2656204e-04, 1.1907238e-12, 1.4297954e-12,
        1.2924976e-10, 2.8937708e-11, 9.7344096e-14, 6.5999772e-13,
        1.2500098e-15, 3.4346459e-09, 1.1532775e-15], dtype=float32),
 array([1.1956422e-05, 7.9956525e-10, 9.9998653e-01, 1.4017409e-06,
        1.4398896e-08, 4.1639157e-12, 4.3190666e-11, 2.8399613e-08,
        1.9454152e-08, 4.0909694e-13, 9.2727926e-12], dtype=float32),
 array([9.9998653e-01, 6.4678659e-07, 2.9132932e-06, 4.0218990e-07,
        4.0837799e-06, 2.4406546e-09, 8.4422830e-08, 5.1561433e-06,
        5.5941055e-12, 1.0488938e-13, 2.4883201e-07], dtype=float32),
 array([9.9990249e-01, 9.3521325e-05, 9.3598260e-08, 6.6723560e-09,
        1.0657799e-08, 2.3611472e-06, 1.1703808e-06, 2.4364988e-07,
        1.0564784e-07, 8.4521253e-12, 8.8053593e-13], dtype=float32),
 array([5.0322815e-05, 7.6318179e-12, 4.7780702e-10, 1.5754531e-11,
        9.9994540e-01, 8.5513375e-07, 3.3595029e-07, 3.0938641e-06,
        4.0547037e-09, 1.9497434e-13, 5.

In [15]:
np.array(roots_prob_ghost).shape

(12, 168)

In [16]:
np.array(vowels_prob_ghost).shape

(12, 11)

In [17]:
len(roots_prob_ghost)

12

In [18]:
cpu_stats()

'memory GB:2.22'

In [19]:
'''
from tqdm import tqdm

roots_id_ghost=[]
roots_prob_ghost=[]

vowels_id_ghost=[]
vowels_prob_ghost=[]

consonants_id_ghost=[]
consonants_prob_ghost=[]

for i in tqdm(range(0, 4)):
    #data = pd.read_parquet(f'../input/bengaliai-cv19/test_image_data_{i}.parquet')
    data = pd.read_parquet(os.path.join(DIR, test_loc+str(i)+'.parquet'))
    dataset = DatasetRetriever(data)
    del data
    data_loader = DataLoader(dataset, batch_size=64, num_workers=0, shuffle=False, sampler=SequentialSampler(dataset))
    
    id_t=[]
    data_t=[]
    
    
    #del data
    
    for x_id,x_data in data_loader:
        id_t.append(x_id)
        data_t.append(x_data)
        
        roots, vowels, consonants = make_prediction(x_data)
        
        roots_prob_ghost.append(roots)
        vowels_prob_ghost.append(vowels)
        consonants_prob_ghost.append(consonants)    
'''

"\nfrom tqdm import tqdm\n\nroots_id_ghost=[]\nroots_prob_ghost=[]\n\nvowels_id_ghost=[]\nvowels_prob_ghost=[]\n\nconsonants_id_ghost=[]\nconsonants_prob_ghost=[]\n\nfor i in tqdm(range(0, 4)):\n    #data = pd.read_parquet(f'../input/bengaliai-cv19/test_image_data_{i}.parquet')\n    data = pd.read_parquet(os.path.join(DIR, test_loc+str(i)+'.parquet'))\n    dataset = DatasetRetriever(data)\n    del data\n    data_loader = DataLoader(dataset, batch_size=64, num_workers=0, shuffle=False, sampler=SequentialSampler(dataset))\n    \n    id_t=[]\n    data_t=[]\n    \n    \n    #del data\n    \n    for x_id,x_data in data_loader:\n        id_t.append(x_id)\n        data_t.append(x_data)\n        \n        roots, vowels, consonants = make_prediction(x_data)\n        \n        roots_prob_ghost.append(roots)\n        vowels_prob_ghost.append(vowels)\n        consonants_prob_ghost.append(consonants)    \n"

# efficient net

## 함수호출

In [20]:

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import cv2
import os
import time, gc
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model, Input
from keras.layers import Dense, Lambda
from math import ceil

# Install EfficientNet
!pip install '../input/kerasefficientnetb3/efficientnet-1.0.0-py3-none-any.whl'
import efficientnet.keras as efn
#!pip install keras_efficientnets
#from keras_efficientnets import EfficientNetB3


Using TensorFlow backend.


Processing /kaggle/input/kerasefficientnetb3/efficientnet-1.0.0-py3-none-any.whl


In [21]:
# 파라미터 설정
HEIGHT = 137
WIDTH = 236
FACTOR = 0.70
HEIGHT_NEW = int(HEIGHT * FACTOR)
WIDTH_NEW = int(WIDTH * FACTOR)
CHANNELS = 3
BATCH_SIZE = 16 #16

# 경로
#DIR = '../input/bengaliai-cv19'

In [22]:

# Image Prep
def resize_image(img, WIDTH_NEW, HEIGHT_NEW):
    # Invert
    img = 255 - img

    # Normalize
    img = (img * (255.0 / img.max())).astype(np.uint8)

    # Reshape
    img = img.reshape(HEIGHT, WIDTH)
    image_resized = cv2.resize(img, (WIDTH_NEW, HEIGHT_NEW), interpolation = cv2.INTER_AREA)

    return image_resized.reshape(-1) 


In [23]:
# Generalized mean pool - GeM
gm_exp = tf.Variable(3.0, dtype = tf.float32)
def generalized_mean_pool_2d(X):
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool

In [24]:
# Create Model

def create_model(input_shape):
    # Input Layer
    input = Input(shape = input_shape)
    
    # Create and Compile Model and show Summary
    x_model = efn.EfficientNetB3(weights = None, include_top = False, input_tensor = input, pooling = None, classes = None)
    
    # UnFreeze all layers
    for layer in x_model.layers:
        layer.trainable = True
    
    # GeM
    lambda_layer = Lambda(generalized_mean_pool_2d)
    lambda_layer.trainable_weights.extend([gm_exp])
    x = lambda_layer(x_model.output)
    
    # multi output
    grapheme_root = Dense(168, activation = 'softmax', name = 'root')(x)
    vowel_diacritic = Dense(11, activation = 'softmax', name = 'vowel')(x)
    consonant_diacritic = Dense(7, activation = 'softmax', name = 'consonant')(x)

    # model
    model = Model(inputs = x_model.input, outputs = [grapheme_root, vowel_diacritic, consonant_diacritic])

    return model


In [25]:
cpu_stats()

'memory GB:2.84'

In [26]:
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, X, batch_size = 16, img_size = (128, 128, 3), *args, **kwargs):
        self.X = X
        self.indices = np.arange(len(self.X))
        self.batch_size = batch_size
        self.img_size = img_size
                    
    def __len__(self):
        return int(ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indices)
        return X
    
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        
        for i, index in enumerate(indices):
            image = self.X[index]
            image = np.stack((image,)*CHANNELS, axis=-1)
            image = image.reshape(-1, HEIGHT_NEW, WIDTH_NEW, CHANNELS)
            
            X[i,] = image
        
        return X

In [27]:
cpu_stats()

'memory GB:2.84'

In [28]:
model5 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model5.load_weights('../input/kerasefficientnetb3/Train1_model_70.h5') 

In [29]:
cpu_stats()

'memory GB:2.97'

In [30]:
#del dataloaders
#del data_generator_test 
#gc.collect() 순서대로 run했으면 필요없는 셀

In [31]:
cpu_stats()

'memory GB:2.97'

In [32]:

from se_resnext import make_loader

# 컬럼명
tgt_cols = ['grapheme_root','vowel_diacritic','consonant_diacritic']

# 예측값 담는 리스트
row_ids, targets = [], []


for i in tqdm(range(0, 4)):
    test_files = []
    
    # Read Parquet file
    df = pd.read_parquet(os.path.join(DIR, test_loc+str(i)+'.parquet'))
    # Get Image Id values
    image_ids = df['image_id'].values 
    # Drop Image_id column
    dataloaders = make_loader(data_folder = df,
                                       batch_size=BATCH_SIZE,
                                       num_workers = 2,
                                       is_shuffle = False)
    df = df.drop(['image_id'], axis = 1)
    
    
    X = []
    for image_id, index in zip(image_ids, range(df.shape[0])):
        test_files.append(image_id)
        X.append(resize_image(df.loc[df.index[index]].values, WIDTH_NEW, HEIGHT_NEW))

        
    del df
    gc.collect()
        
    # Data_Generator
    data_generator_test = TestDataGenerator(X, batch_size = BATCH_SIZE, img_size = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
    preds5 = model5.predict_generator(data_generator_test, verbose = 1)

    
    del dataloaders
    del data_generator_test
    #del model5 이거 주석풀면 preds5 = 라인에서 뒤짐
    gc.collect()
    cpu_stats()
    
    for i, image_id in zip(range(len(test_files)), test_files):
        
        for subi, col in zip(range(len(preds5)), tgt_cols):
            sub_preds5 = preds5[subi]

            # Set Prediction with average of 5 predictions
            row_ids.append(str(image_id)+'_'+col)
            sub_pred_value =sub_preds5[i] 
            targets.append(sub_pred_value)


  0%|          | 0/4 [00:00<?, ?it/s]

1/1 [==============================] - 4s 4s/step


 25%|██▌       | 1/4 [00:09<00:28,  9.47s/it]

1/1 [==============================] - 0s 44ms/step


 50%|█████     | 2/4 [00:12<00:15,  7.53s/it]

1/1 [==============================] - 0s 46ms/step


 75%|███████▌  | 3/4 [00:15<00:06,  6.14s/it]

1/1 [==============================] - 0s 44ms/step


100%|██████████| 4/4 [00:18<00:00,  4.56s/it]


In [33]:
cpu_stats()

'memory GB:3.31'

In [34]:
#row_ids, targets = [], [] 나중에는 지워서 램 절약

In [35]:
len(targets)

36

In [36]:
roots_prob_effi=[]
vowels_prob_effi=[]
consonants_prob_effi=[]

for i in range(0, len(targets)//3):
    roots_prob_effi.append(targets[i*3])
    
for i in range(0, len(targets)//3):
    vowels_prob_effi.append(targets[i*3+1])
    
for i in range(0, len(targets)//3):
    consonants_prob_effi.append(targets[i*3+2])


'''
xx=[0,3,6,9,12,15,18,21,24,27,30,33]
yy=[1,4,7,10,13,16,19,22,25,28,31,34]
zz=[2,5,8,11,14,17,20,23,26,29,32,35]

for i in xx:
    roots_prob_effi.append(targets[i])
    
for i in yy:
    vowels_prob_effi.append(targets[i])
    
for i in zz:
    consonants_prob_effi.append(targets[i])
'''


'\nxx=[0,3,6,9,12,15,18,21,24,27,30,33]\nyy=[1,4,7,10,13,16,19,22,25,28,31,34]\nzz=[2,5,8,11,14,17,20,23,26,29,32,35]\n\nfor i in xx:\n    roots_prob_effi.append(targets[i])\n    \nfor i in yy:\n    vowels_prob_effi.append(targets[i])\n    \nfor i in zz:\n    consonants_prob_effi.append(targets[i])\n'

In [37]:
np.array(roots_prob_effi).shape

(12, 168)

In [38]:
cpu_stats()

'memory GB:3.31'

In [39]:
np.array(roots_prob_effi).shape

(12, 168)

In [40]:
#np.array(vowels_prob_effi).shape

# se-resnet

In [41]:
from __future__ import print_function, division, absolute_import
from matplotlib import pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms as torchtransforms
import cv2
import torch.nn as nn
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import time
from collections import OrderedDict
import math
from torch.utils import model_zoo

In [42]:
cpu_stats()

'memory GB:3.31'

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
modelpath = "/kaggle/input/se-resnext50-32x4d-fold2/se_resnext50_32x4d_fold2.pkl"
#modelpath = "/kaggle/input/se-resnext50-32x4d-fold3/se_resnext50_32x4d_fold3.pkl"
root_path="/kaggle/input/bengaliai-cv19"

cuda:0


In [44]:
cpu_stats()

'memory GB:3.31'

In [45]:
def getmodeleval(model, dataloaders):
    model.eval()
    #tbar = tqdm(dataloaders)
    results = []
    pathes=[]

    alllogit1 = []
    alllogit2 = []
    alllogit3 = []
    for path, img in dataloaders:
        img = img.to(device)
        pathes.extend(path)
        with torch.no_grad():
            output = model(img)
        logit1, logit2, logit3 = output[:,: 168],\
                                    output[:,168: 168+11],\
                                    output[:,168+11:]
        logit1 = F.softmax(logit1, dim=1).cpu().numpy()  # softmax
        logit2 = F.softmax(logit2, dim=1).cpu().numpy()
        logit3 = F.softmax(logit3, dim=1).cpu().numpy()
        alllogit1.extend(logit1.tolist())
        alllogit2.extend(logit2.tolist())
        alllogit3.extend(logit3.tolist())
    alllogit1 = np.array(alllogit1)
    alllogit2 = np.array(alllogit2)
    alllogit3 = np.array(alllogit3)
    results.append(alllogit1)
    results.append(alllogit2)
    results.append(alllogit3)
    return pathes, results

In [46]:
from se_resnext import make_loader
from se_resnext import se_resnext50_32x4d
model6 = se_resnext50_32x4d(pretrained=None)
model6.avg_pool = nn.AdaptiveAvgPool2d(1)
model6.last_linear = nn.Linear(model6.last_linear.in_features, 186)
modelvalue = torch.load(modelpath, map_location='cpu')
#modelvalue = torch.load(modelpath, map_location='cuda:0')
newmodelvalue = {}
for kv in modelvalue:
    newmodelvalue[kv[4:]]=modelvalue[kv]        
model6.load_state_dict(newmodelvalue)
model6 = model6.to(device)

In [47]:
cpu_stats()

'memory GB:3.36'

In [48]:
#del dataloaders
#gc.collect()

In [49]:
cpu_stats()

'memory GB:3.36'

In [50]:
#del dataloaders
#del df 램 용량 많이 먹음
gc.collect()
cpu_stats()

'memory GB:3.36'

In [51]:
gc.collect()
cpu_stats()

'memory GB:3.36'

In [52]:
roots_prob_res=[]
vowels_prob_res=[]
consonants_prob_res=[]

# Create Submission File
tgt_cols = ['grapheme_root','vowel_diacritic','consonant_diacritic']

# Create Predictions
row_ids, targets = [], []

# Loop through Test Parquet files (X)
for i in tqdm(range(0, 4)):
    # Test Files Placeholder
    test_files = []

    # Read Parquet file
    df = pd.read_parquet(os.path.join(DIR, test_loc+str(i)+'.parquet'))
    # Get Image Id values
    image_ids = df['image_id'].values 
    # Drop Image_id column
    dataloaders = make_loader(data_folder = df,
                                       batch_size=8,
                                       num_workers = 0, # 원본 : 2
                                       is_shuffle = False)
    df = df.drop(['image_id'], axis = 1)
    
    
    
    # 삽입한 코드  
    
    X = []
    for image_id, index in zip(image_ids, range(df.shape[0])):
        test_files.append(image_id)
        X.append(resize_image(df.loc[df.index[index]].values, WIDTH_NEW, HEIGHT_NEW))
    
    # 삽입한 코드 end
    
    

    data_generator_test = TestDataGenerator(X, batch_size = BATCH_SIZE, img_size = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
    
    pathes6, preds6 = getmodeleval(model6, dataloaders)
    
    
    for kk in preds6[0]:
        roots_prob_res.append(kk)
    
    for kk in preds6[1]:
        vowels_prob_res.append(kk)
    
    for kk in preds6[2]:
        consonants_prob_res.append(kk)
        
    del dataloaders
    del df
    gc.collect()
    cpu_stats()
    
    # Cleanup
    #del df
    #gc.collect()

100%|██████████| 4/4 [00:11<00:00,  2.96s/it]


In [53]:
# 데이터 빼내기
#roots_prob_ghost_q 대신 roots_prob_ghost 사용하므로 필요없는 셀이다
'''
roots_prob_ghost_q=[]
vowels_prob_ghost_q=[]
consonants_prob_ghost_q=[]


for i in roots_prob_ghost:
    for j in i:
        roots_prob_ghost_q.append(j)
        
for i in vowels_prob_ghost:
    for j in i:
        vowels_prob_ghost_q.append(j)

for i in consonants_prob_ghost:
    for j in i:
        consonants_prob_ghost_q.append(j)
'''

'\nroots_prob_ghost_q=[]\nvowels_prob_ghost_q=[]\nconsonants_prob_ghost_q=[]\n\n\nfor i in roots_prob_ghost:\n    for j in i:\n        roots_prob_ghost_q.append(j)\n        \nfor i in vowels_prob_ghost:\n    for j in i:\n        vowels_prob_ghost_q.append(j)\n\nfor i in consonants_prob_ghost:\n    for j in i:\n        consonants_prob_ghost_q.append(j)\n'

In [54]:
'''
데이터 셋

roots_prob_ghost
vowels_prob_ghost
consonants_prob_ghost

roots_prob_ghost_q
vowels_prob_ghost_q
consonants_prob_ghost_q

roots_prob_effi
vowels_prob_effi
consonants_prob_effi

roots_prob_res
vowels_prob_res
consonants_prob_res
'''

'\n데이터 셋\n\nroots_prob_ghost\nvowels_prob_ghost\nconsonants_prob_ghost\n\nroots_prob_ghost_q\nvowels_prob_ghost_q\nconsonants_prob_ghost_q\n\nroots_prob_effi\nvowels_prob_effi\nconsonants_prob_effi\n\nroots_prob_res\nvowels_prob_res\nconsonants_prob_res\n'

In [55]:
np.array(roots_prob_res).shape

(12, 168)

In [56]:
np.array(roots_prob_effi).shape

(12, 168)

In [57]:
np.array(roots_prob_ghost).shape

(12, 168)

# denseNET

In [58]:
import sys

import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler
from torchvision.models.densenet import densenet121

sys.path.insert(0, "/kaggle/input/ghostnetbengali")

from ghost_net import ghost_net

In [59]:
IMAGE_SIZE = 137, 236


class DatasetRetriever(Dataset):
    
    def __init__(self, df):
        self.image_ids = df.iloc[:, 0].tolist()
        self.images = torch.from_numpy(255 - df[[str(x) for x in range(32332)]].values)

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        img = self.images[idx]
        img = img.view(*IMAGE_SIZE)
        img = img.to(torch.float32) / 255.
        img = img.unsqueeze(0)
        img = img.repeat(3, 1, 1)
        img_id = self.image_ids[idx]        
        return img_id, img
    
class Predictor:
    def __init__(self, model):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device, dtype=torch.float32);
        self.model.eval()
        print(f'Model prepared. Device is {self.device}')
    
    def predict(self, inputs):
        inputs = inputs.to(self.device, dtype=torch.float32)
        with torch.no_grad():
            outputs = self.model(inputs)
        return outputs

    def load(self, path):
        checkpoint = torch.load(path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])

In [60]:
predictors = []

for i, model_path in enumerate([
'/kaggle/input/ghostnetbengali/densenet121-checkpoint.pt',
]):
    predictor = Predictor(densenet121(num_classes=168+11+7))
    predictor.load(model_path)
    predictors.append(predictor)

predictors_count = len(predictors)


def predict_to_numpy(predict):
    return torch.nn.functional.softmax(predict, dim=1).data.cpu().numpy()

def make_prediction(images):
    outputs = 0
    for predictor in predictors:
        outputs += predictor.predict(images) / predictors_count

    roots = predict_to_numpy(outputs[:,:168])
    vowels = predict_to_numpy(outputs[:,168:168+11])
    consonants = predict_to_numpy(outputs[:,168+11:])
    return roots, vowels, consonants

Model prepared. Device is cuda:0


In [61]:
%%time

target = []
row_id = []

roots_id_dense=[]
roots_prob_dense=[]

vowels_id_dense=[]
vowels_prob_dense=[]

consonants_id_dense=[]
consonants_prob_dense=[]

for i in range(4):
    data = pd.read_parquet(os.path.join(DIR, test_loc+str(i)+'.parquet'))
    #data = pd.read_parquet(f'../input/bengaliai-cv19/test_image_data_{i}.parquet')
    dataset = DatasetRetriever(data)
    data_loader = DataLoader(dataset, batch_size=256, num_workers=4, shuffle=False, sampler=SequentialSampler(dataset))

    for idx, (image_ids, images) in tqdm(enumerate(data_loader), total=len(data_loader)):

        roots, vowels, consonants = make_prediction(images)

        #for image_id, root, vowel, consonant in zip(image_ids, roots, vowels, consonants):
        for root, vowel, consonant in zip(roots, vowels, consonants):
            #원본
            #row_id.append(image_id + '_consonant_diacritic')
            #target.append(consonant)
            #row_id.append(image_id + '_grapheme_root')
            #target.append(root)
            #row_id.append(image_id + '_vowel_diacritic')
            #target.append(vowel)
            
            
            
            #roots_prob_ghost.append(roots)
            #vowels_prob_ghost.append(vowels)
            #consonants_prob_ghost.append(consonants)             
            
            
            roots_prob_dense.append(root)
            vowels_prob_dense.append(vowel)
            consonants_prob_dense.append(consonant)    
            
    del data
    del dataset
    del data_loader
    collected = gc.collect()

100%|██████████| 1/1 [00:00<00:00,  7.25it/s]

CPU times: user 12.1 s, sys: 2.31 s, total: 14.4 s
Wall time: 12.3 s


In [62]:
root_all=[]
vowels_all=[]
consonants=[]

#코드 수정해서 roots_prob_ghost_q 대신 roots_prob_ghost 사용

for i in range(len(roots_prob_ghost)):
    xxxx=(roots_prob_ghost[i] +roots_prob_effi[i] + roots_prob_res[i]*2+roots_prob_dense[i])/5
    #xxxx=(roots_prob_ghost[i] +roots_prob_effi[i] + roots_prob_res[i])/3
    #xxxx=(roots_prob_ghost[i] + roots_prob_res[i])/2
    root_all.append(xxxx)
    
for i in range(len(roots_prob_ghost)):
    xxxx=(vowels_prob_ghost[i] +vowels_prob_effi[i] + vowels_prob_res[i]*2+vowels_prob_dense[i])/5
    #xxxx=(vowels_prob_ghost[i] +vowels_prob_effi[i] + vowels_prob_res[i])/3
    #xxxx=(vowels_prob_ghost[i] + vowels_prob_res[i])/2
    vowels_all.append(xxxx)
    
for i in range(len(roots_prob_ghost)):
    xxxx=(consonants_prob_ghost[i] +consonants_prob_effi[i] + consonants_prob_res[i]*2+consonants_prob_dense[i])/5
    #xxxx=(consonants_prob_ghost[i] +consonants_prob_effi[i] + consonants_prob_res[i])/3
    #xxxx=(consonants_prob_ghost[i] + consonants_prob_res[i])/2
    consonants.append(xxxx)

In [63]:
len(root_all)

12

In [64]:
len(vowels_all)

12

In [65]:
len(consonants)

12

In [66]:
np.argmax(root_all[0])

3

In [67]:
targets=[]

for i in range(len(root_all)):
    targets.append(np.argmax(root_all[i]))
    targets.append(np.argmax(vowels_all[i]))
    targets.append(np.argmax(consonants[i]))

In [68]:
# Create Submission File
tgt_cols = ['grapheme_root','vowel_diacritic','consonant_diacritic']

id_get=[]

# Loop through Test Parquet files (X)
for i in tqdm(range(0, 4)):
    # Test Files Placeholder
    test_files = []

    # Read Parquet file
    df = pd.read_parquet(os.path.join(DIR, test_loc+str(i)+'.parquet'))
    # Get Image Id values
    [id_get.append(nn) for nn in df['image_id'].values]

100%|██████████| 4/4 [00:10<00:00,  2.56s/it]


In [69]:
id_list=[]
for i in id_get:
    id_list.append(i +"_"+tgt_cols[0])
    id_list.append(i +"_"+ tgt_cols[1])
    id_list.append(i +"_"+ tgt_cols[2])

In [70]:
submission=pd.DataFrame(targets,id_list).reset_index()
submission.columns=["row_id","target"]

In [71]:
submission.to_csv("submission.csv",index=False)

In [72]:
submission

,row_id,target
0,Test_0_grapheme_root,3
1,Test_0_vowel_diacritic,0
2,Test_0_consonant_diacritic,0
3,Test_1_grapheme_root,93
4,Test_1_vowel_diacritic,2
5,Test_1_consonant_diacritic,0
6,Test_2_grapheme_root,19
7,Test_2_vowel_diacritic,0
8,Test_2_consonant_diacritic,0
9,Test_3_grapheme_root,115
